<a href="https://colab.research.google.com/github/hojuna/Black-box_Optimization_dacon/blob/main/test_TheilSenRegressor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Reduce Memory Usage
# reference : https://www.kaggle.com/code/arjanso/reducing-dataframe-memory-size-by-65 @ARJANGROEN

def reduce_memory_usage(df):

    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))

    for col in df.columns:
        col_type = df[col].dtype.name
        if ((col_type != 'datetime64[ns]') & (col_type != 'category')):
            if (col_type != 'object'):
                c_min = df[col].min()
                c_max = df[col].max()

                if str(col_type)[:3] == 'int':
                    if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                        df[col] = df[col].astype(np.int8)
                    elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                        df[col] = df[col].astype(np.int16)
                    elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                        df[col] = df[col].astype(np.int32)
                    elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                        df[col] = df[col].astype(np.int64)

                else:
                    if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                        df[col] = df[col].astype(np.float16)
                    elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                        df[col] = df[col].astype(np.float32)
                    else:
                        pass
            else:
                df[col] = df[col].astype('category')
    mem_usg = df.memory_usage().sum() / 1024**2
    print("Memory usage became: ",mem_usg," MB")

    return df

In [3]:
import pandas as pd
import numpy as np
import joblib
import os
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV,cross_val_score
from sklearn.model_selection import RandomizedSearchCV


# Load the data
train_df_org = pd.read_csv('/content/drive/MyDrive/dacon/2024_8_data/train.csv')
test_df = pd.read_csv('/content/drive/MyDrive/dacon/2024_8_data/test.csv')

In [4]:
train_df_org_reduced = reduce_memory_usage(train_df_org)
test_df_reduced = reduce_memory_usage(test_df)

Memory usage of dataframe is 3.98 MB
Memory usage became:  2.38531494140625  MB
Memory usage of dataframe is 0.46 MB
Memory usage became:  0.27829742431640625  MB


In [ ]:
# df_shuffled= train_df_org
df_shuffled= train_df_org_reduced

# 균등하게 나누기 위해 임의로 데이터를 8:2로 분할
split_index = int(len(df_shuffled) * 0.8)
train_df, valiation_df = df_shuffled[:split_index], df_shuffled[split_index:]


In [ ]:

def val_score(model, valiation_df):
  id_y=pd.DataFrame()

  top_5_percent_valiation_df = valiation_df.sort_values(by='y').reset_index(drop=True)[-int(0.05 * len(valiation_df)):]
  #id만 분리 valiation_df, y값 제거
  X = valiation_df.iloc[:, 1:-1]
  id_y['ID']=valiation_df.iloc[:,0]

  #valiation_df y값 예측
  y_pred=model.predict(X)
  id_y['y_pred']=y_pred

  # print(id_x.head)
  # #id 복원

  #상위 10% valiation_df 구하기
  top_10_percent_pred_valiation=id_y.sort_values(by='y_pred').reset_index(drop=True)[-int(0.10 * len(id_y)):]

  # ID 값에서 공백 제거
  top_5_percent_valiation_df['ID'] = top_5_percent_valiation_df['ID'].str.strip()
  top_10_percent_pred_valiation['ID'] = top_10_percent_pred_valiation['ID'].str.strip()

  #top_5_percent_valiation_df 중  상위 10%에 포함되는 비율을 구해서 반환
  return top_5_percent_valiation_df['ID'].isin(top_10_percent_pred_valiation['ID']).sum()/len(top_5_percent_valiation_df)

In [5]:

X = train_df_org_reduced.iloc[:, 1:-1]  # Features
y = train_df_org_reduced['y']  # Target

In [6]:
from sklearn.linear_model import TheilSenRegressor


model = joblib.load("/content/drive/MyDrive/dacon/model/tuning_model/theilsen_model.pkl")
model.fit(X, y)

TheilSenRegressor(max_iter=100, max_subpopulation=20000, tol=0.0001)

In [7]:


# Predict on test data
X_test = test_df_reduced.iloc[:, 1:]
y_pred = model.predict(X_test)

# Identify top 5% of predicted values
threshold = np.percentile(y_pred, 95)
top_5_percent_mask = y_pred >= threshold

# Create submission file
submission_df = pd.read_csv("/content/drive/MyDrive/dacon/2024_8_data/submission/sample_submission.csv")
submission_df['y'] = y_pred
submission_df.to_csv("/content/drive/MyDrive/dacon/2024_8_data/submission/theilsen_submission.csv", index=False)

print(f"Top 5% threshold: {threshold:.4f}")
print(f"Number of samples in top 5%: {sum(top_5_percent_mask)}")

Top 5% threshold: 93.5123
Number of samples in top 5%: 250
